# LinearModel

In [ ]:
medium = ""

In [ ]:
import SparseArrays: sparse
import NBInclude: @nbinclude
@nbinclude("../Alpha.ipynb")
@nbinclude("EnsembleInputs.ipynb");

In [ ]:
function save_linear_model(metric, medium)
    alphas = get_ensemple_alphas(metric, medium)
    β = regress(get_features("test", metric, medium, alphas)..., metric)
    name = "$medium/Linear/$metric"
    @info "$medium $metric linear model $β $alphas"
    write_params(Dict("β" => β), name, true)

    # inference
    M = SparseArrays.sparse(Int32[], Int32[], Float32[], num_users(), num_items(medium))
    for split in ["test", "negative"]
        X = [get_raw_split(split, medium, [:userid], a).alpha for a in alphas]
        if metric in ["watch", "plantowatch"]
            push!(X, fill(1.0f0 / num_items(medium), length(X[1])))
        elseif metric == "drop"
            push!(X, fill(1.0f0, length(X[1])), fill(0.0f0, length(X[1])))
        end
        X = hcat(X...)
        y = X * β
        u = get_raw_split(split, medium, [:userid], nothing).userid
        a = get_raw_split(split, medium, [:itemid], nothing).itemid
        M .+= SparseArrays.sparse(u .+ 1, a .+ 1, y, num_users(), num_items(medium))
    end
    function model(userids, itemids)
        r = zeros(Float32, length(userids))
        @tprogress Threads.@threads for i = 1:length(userids)
            r[i] = M[userids[i]+1, itemids[i]+1]
        end
        r
    end

    write_alpha(model, medium, name, ["test", "negative"])
    print_losses(alphas, metric, medium, ["test"])
end;

In [ ]:
for metric in ALL_METRICS
    save_linear_model(metric, medium)
end;